# Recurrent Neural Network with Long Short-Term Memory layers

In [1]:
import numpy as np
import nolds
import scipy.io
import mne
import pandas as pd
import seaborn as sns
import pyqtgraph as pg
from scipy.io import loadmat
from scipy import stats
import matplotlib.pyplot as plt
import antropy as ent
from attractors import Attractor
from antropy import higuchi_fd
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.computation import RQAComputation
from pyrqa.time_series import TimeSeries
from pyrqa.result import RQAResult
from pyrqa.opencl import OpenCL
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.image_generator import ImageGenerator
import numpy as np
import matplotlib.pyplot as plt
from antropy import higuchi_fd

In [2]:
# Load data
data = loadmat('/home/vincent/AAA_projects/MVCS/Neuroscience/downsampled/EEG_DS_Struct_0101.mat')

In [4]:
DSamp = data['DSamp']

triggers = DSamp[0][0][0]
EEGdata = DSamp[0][0][1]
fs = DSamp[0][0][2][0][0] 
fsOld = DSamp[0][0][3][0][0] 
time = DSamp[0][0][4][0]
label = DSamp[0][0][5]
nchan = DSamp[0][0][6][0][0]
rate = DSamp[0][0][7][0][0]
npt = DSamp[0][0][8][0][0]
Subj = DSamp[0][0][9][0]
ptrackerPerf = DSamp[0][0][10]
ptrackerTime = DSamp[0][0][11]
ptrackerfs = DSamp[0][0][12][0][0]

In [13]:
TESdata = triggers  # Assuming TES data is stored here

# Transpose your data if necessary
# This step will depend on how your data is structured. If it's already in the correct shape, you don't need to transpose it
TESdata = TESdata.T

# Check the shape of your TES data
print(f"Shape of TES data before reshaping: {TESdata.shape}")

# Ensure the correct reshaping parameters
print(f"n_samples: {n_samples}, n_steps: {n_steps}")

# Reshape your data
# The reshape operation may need to change depending on the structure of your data and the needs of your model
try:
    TESdata_reshaped = TESdata[:n_samples*n_steps].reshape(-1, n_steps, TESdata.shape[1])
except ValueError as e:
    print(f"Error during reshaping: {e}")

# Convert to float32
TESdata_reshaped = TESdata_reshaped.astype('float32')

# Print out shape
print("TESdata_reshaped shape: ", TESdata_reshaped.shape)

# Then use TESdata_reshaped as input to your model
model.fit(TESdata_reshaped, label_reshaped, batch_size=64, epochs=10)

Shape of TES data before reshaping: (24, 1)
n_samples: 4227, n_steps: 1000
Error during reshaping: cannot reshape array of size 24 into shape (1000,1)


NameError: name 'TESdata_reshaped' is not defined

In [14]:
# Flatten the array to 1D
flattened_label = np.concatenate([item[0] for item in label]).ravel()

# Encode the labels
label_encoded = label_encoder.fit_transform(flattened_label)

# Reshape to 2D
label_reshaped = label_encoded.reshape(-1, 1).astype('float32')

In [15]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Flatten the array to 1D and extract the string from the label
flattened_label = np.concatenate([item[0] for item in label]).ravel()

# Encode the labels
label_encoded = label_encoder.fit_transform(flattened_label)

# Reshape to match the expected shape
label_reshaped = label_encoded.reshape(-1, 1).astype('float32')

In [16]:
EEGdata_reshaped = EEGdata_reshaped.astype('float32')
label_reshaped = label_reshaped.astype('float32')

In [17]:
print("EEGdata_reshaped shape: ", EEGdata_reshaped.shape)
print("label_reshaped shape: ", label_reshaped.shape)

EEGdata_reshaped shape:  (4227, 1000, 35)
label_reshaped shape:  (35, 1)


In [18]:
print("EEGdata contains NaN: ", np.isnan(EEGdata_reshaped).any())
print("EEGdata contains Inf: ", np.isinf(EEGdata_reshaped).any())
print("label contains NaN: ", np.isnan(label_reshaped).any())
print("label contains Inf: ", np.isinf(label_reshaped).any())

EEGdata contains NaN:  False
EEGdata contains Inf:  False
label contains NaN:  False
label contains Inf:  False


In [19]:
n_channels = EEGdata_reshaped.shape[2]
label_reshaped = np.repeat(label_encoded[:n_samples, np.newaxis], n_steps, axis=1).reshape(-1, n_steps, 1)

In [20]:
# Load your data
DSamp = data['DSamp']
EEGdata = DSamp[0][0][1]

# Transpose your data
EEGdata = EEGdata.T

# Number of timesteps you want for each sample
n_steps = 1000

# Number of total samples you will have after the segmentation
n_samples = EEGdata.shape[0] // n_steps

# Reshape your data
EEGdata_reshaped = EEGdata[:n_samples*n_steps].reshape(-1, n_steps, EEGdata.shape[1])

# We want to predict the future state of the EEG reading, so we shift the EEG readings
# to create the "labels"
future_step = 1  # Change this to predict more steps into the future
EEGdata_future = EEGdata[future_step:n_samples*n_steps+future_step].reshape(-1, n_steps, EEGdata.shape[1])

EEGdata_reshaped = EEGdata_reshaped.astype('float32')
EEGdata_future = EEGdata_future.astype('float32')

# Print out shapes
print("EEGdata_reshaped shape: ", EEGdata_reshaped.shape)
print("EEGdata_future shape: ", EEGdata_future.shape)

# Define your LSTM model
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(n_steps, EEGdata.shape[1])))
model.add(Dropout(0.5))
model.add(LSTM(32, return_sequences=True))
model.add(Dense(EEGdata.shape[1], activation='linear'))  # Predict the future state of all channels

# Compile and train your model
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(EEGdata_reshaped[:-1], EEGdata_future[:-1], batch_size=64, epochs=10)  # We subtract 1 from the shapes to match their sizes

EEGdata_reshaped shape:  (4227, 1000, 35)
EEGdata_future shape:  (4227, 1000, 35)


NameError: name 'Sequential' is not defined